In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load the datasets
joint_angles_df = pd.read_csv('/mnt/data/joint_angles.csv')
poi_metrics_df = pd.read_csv('/mnt/data/poi_metrics.csv')

In [ ]:
import pandas as pd
import numpy as np

# Load the datasets
joint_angles_df =  pd.read_csv(r"C:\Users\conne\Documents\GitHub\openbiomechanics\baseball_pitching\data\full_sig\joint_angles.csv")
poi_metrics_df =  pd.read_csv(r"C:\Users\conne\Documents\GitHub\openbiomechanics\baseball_pitching\data\poi\poi_metrics.csv")

# Filter the relevant columns from both datasets before merging
filtered_joint_angles_df = joint_angles_df.iloc[:, :-5]
filtered_poi_metrics_df = poi_metrics_df[['session_pitch', 'session', 'pitch_speed_mph']]

# Merge the two datasets based on the 'session_pitch' variable
merged_df = pd.merge(filtered_joint_angles_df, filtered_poi_metrics_df, on='session_pitch')

# Filter the data to include only the specified pitches: 1097_1, 1097_2, 1097_3
df_filtered_1097 = merged_df[merged_df['session_pitch'].isin(['1097_1', '1097_2', '1097_3'])]

# Identify the slowest pitch in the session based on pitch speed
slowest_pitch = df_filtered_1097.loc[df_filtered_1097['pitch_speed_mph'].idxmin()]['session_pitch']

# Initialize an empty dataframe to store the trimmed data
df_trimmed_1097 = pd.DataFrame()

# Find unique pitch sessions for the filtered data
unique_pitches_1097 = df_filtered_1097['session_pitch'].unique()

# Initialize variable to store the length of the shortest time series
min_length_1097 = float('inf')

# Loop through each unique pitch to trim and crop the data
for pitch in unique_pitches_1097:
    df_pitch = df_filtered_1097[df_filtered_1097['session_pitch'] == pitch]
    
    # Find the time of the peak leg lift
    peak_leg_lift_time = df_pitch['pkh_time'].iloc[0]
    
    # Trim the series to start at the peak of the leg lift
    df_trim = df_pitch[df_pitch['time'] >= peak_leg_lift_time]
    
    # Update the 'time' variable so it starts from 0 and increments by 1
    df_trim['time'] = range(len(df_trim))
    
    # Update the minimum length
    min_length_1097 = min(min_length_1097, len(df_trim))
    
    # Append to the trimmed dataframe
    df_trimmed_1097 = pd.concat([df_trimmed_1097, df_trim])

# Make each time series the same length by trimming to the shortest one
df_final_1097 = pd.DataFrame()
for pitch in unique_pitches_1097:
    df_pitch = df_trimmed_1097[df_trimmed_1097['session_pitch'] == pitch]
    df_final_1097 = pd.concat([df_final_1097, df_pitch.iloc[:min_length_1097]])

# Show the first few rows of the final trimmed and cropped dataframe
trimmed_dfs = df_final_1097

scores = {}
        
for col in trimmed_dfs.columns[3:-1]:
            
    sums =  0
            
    for t in range(0, min_length_1097):
            
        slowest_value = trimmed_dfs[(trimmed_dfs['session_pitch'] == slowest_pitch) & (trimmed_dfs['time'] == t)][col].iloc[0]
        #slowest_value = trimmed_dfs.query(f"session_pitch == '{slowest_pitch}' and time == {t}")[col].iloc[]
        other_values = trimmed_dfs[(trimmed_dfs['session_pitch'] != slowest_pitch) & (trimmed_dfs['time'] == t)][col]
        average_value = sum(other_values)/(len(trimmed_dfs['session_pitch'].unique()) - 1)
        diff = abs(slowest_value - average_value)
        sums += (diff * ((min_length_1097 - t)/min_length_1097))
    
    
    multiplier = 0.5  # Default multiplier
    if any(keyword in col for keyword in ['ankle', 'knee', 'hip', 'pelvis']):
        multiplier = 1.5
    elif any(keyword in col for keyword in ['torso', 'shoulder']):
        multiplier = 1.0
            
    scores[col] = (sums/min_length_1097) * multiplier

In [ ]:
sorted_scores_dict = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}

# Pick the top 3 variables
top_3_vars = list(sorted_scores_dict.keys())[:3]

# Plot the time series for the top 3 variables
for var in top_3_vars:
    plt.figure(figsize=(10, 6))
    for pitch in df_filtered_1097['session_pitch'].unique():
        plt.plot(df_filtered_1097[df_filtered_1097['session_pitch'] == pitch]['time'],
                 df_filtered_1097[df_filtered_1097['session_pitch'] == pitch][var],
                 label=f"Pitch: {pitch.split('_')[1]}")
    plt.title(f"Time Series for {var} in Session 1097")
    plt.xlabel("Time")
    plt.ylabel(var)
    plt.legend()
    plt.show()

In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load the datasets
joint_angles_df = pd.read_csv('/mnt/data/joint_angles.csv')
poi_metrics_df = pd.read_csv('/mnt/data/poi_metrics.csv')

# Merge the datasets
joint_angles_df = joint_angles_df.merge(poi_metrics_df[['session_pitch', 'pitch_speed_mph']], on='session_pitch', how='inner')

# Remove unnecessary columns
joint_angles_df = joint_angles_df.drop(joint_angles_df.columns[-6:], axis=1)

# Group the data by session
grouped_by_session = joint_angles_df.groupby(joint_angles_df['session_pitch'].str.split('_').str[0])

# Define a function to perform DTW algorithm from scratch
def dtw_distance(s1, s2):
    DTW = {}
    
    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist = abs(s1[i] - s2[j])
            DTW[(i, j)] = dist + min(DTW[(i-1, j)], DTW[(i, j-1)], DTW[(i-1, j-1)])
            
    return DTW[len(s1)-1, len(s2)-1]

# Filter for players with a slowest pitch that is more than 1 std dev below the average
filtered_sessions = []
for session, session_data in grouped_by_session:
    avg_speed = session_data.groupby('session_pitch')['pitch_speed_mph'].first().mean()
    std_speed = session_data.groupby('session_pitch')['pitch_speed_mph'].first().std()
    slowest_pitch = session_data.groupby('session_pitch')['pitch_speed_mph'].first().idxmin()
    slowest_speed = session_data.loc[session_data['session_pitch'] == slowest_pitch, 'pitch_speed_mph'].iloc[0]
    
    if slowest_speed <= avg_speed - std_speed:
        filtered_sessions.append(session)

# Pick a random session to analyze
random_session = np.random.choice(filtered_sessions)

# Filter the data for this session
random_session_data = joint_angles_df[joint_angles_df['session_pitch'].str.contains(random_session)]

# Identify the 'slowest' pitch in the session
slowest_pitch = random_session_data.groupby('session_pitch')['pitch_speed_mph'].first().idxmin()
slowest_pitch_data = random_session_data[random_session_data['session_pitch'] == slowest_pitch]

# Prepare a dictionary to store DTW distances for each biomechanical variable in the session
session_dtw_dict = {}

# Loop through each biomechanical variable
for col in random_session_data.columns[3:-1]:
    dtw_distances = []
    for pitch in random_session_data['session_pitch'].unique():
        if pitch == slowest_pitch:
            continue
        slowest_pitch_series = slowest_pitch_data[col].values
        current_pitch_series = random_session_data[random_session_data['session_pitch'] == pitch][col].values
        distance = dtw_distance(slowest_pitch_series, current_pitch_series)
        dtw_distances.append(distance)
    avg_dtw_distance = np.mean(dtw_distances)
    session_dtw_dict[col] = avg_dtw_distance

# Sort the dictionary by DTW distance to identify the variables with the most significant deviations
sorted_dtw_dict = {k: v for k, v in sorted(session_dtw_dict.items(), key=lambda item: item[1], reverse=True)}

# Pick the top 3 variables
top_3_vars = list(sorted_dtw_dict.keys())[:3]

# Plot the time series for the top 3 variables
for var in top_3_vars:
    plt.figure(figsize=(10, 6))
    for pitch in random_session_data['session_pitch'].unique():
        plt.plot(random_session_data[random_session_data['session_pitch'] == pitch]['time'],
                 random_session_data[random_session_data['session_pitch'] == pitch][var],
                 label=f"Pitch: {pitch.split('_')[1]}")
    plt.title(f"Time Series for {var} in Session {random_session}")
    plt.xlabel("Time")
    plt.ylabel(var)
    plt.legend()
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import warnings
from heapq import nlargest
from collections import Counter
warnings.filterwarnings('ignore')

# Load the datasets
joint_angles_df =  pd.read_csv(r"C:\Users\conne\Documents\GitHub\openbiomechanics\baseball_pitching\data\full_sig\joint_angles.csv")
poi_metrics_df =  pd.read_csv(r"C:\Users\conne\Documents\GitHub\openbiomechanics\baseball_pitching\data\poi\poi_metrics.csv")

# Filter the relevant columns from both datasets before merging
filtered_joint_angles_df = joint_angles_df.iloc[:, :-5]
filtered_poi_metrics_df = poi_metrics_df[['session_pitch', 'session', 'pitch_speed_mph']]

# Merge the two datasets based on the 'session_pitch' variable
merged_df = pd.merge(filtered_joint_angles_df, filtered_poi_metrics_df, on='session_pitch')

# Re-initialize the dictionary to store the 'root cause' variables for each pitcher
root_cause_dict = {}

# Loop through each unique session (pitcher)
for session in merged_df['session'].unique():
    
    # Filter data for the current session (pitcher)
    session_data = merged_df[merged_df['session'] == session]
    
    # Identify the slowest pitch in the session based on pitch speed
    slowest_pitch = session_data.loc[session_data['pitch_speed_mph'].idxmin()]['session_pitch']
    
    # Calculate the mean and standard deviation for pitch speeds, excluding the slowest pitch
    mean_speed = np.mean(session_data[session_data['session_pitch'] != slowest_pitch]['pitch_speed_mph'])
    std_speed = np.std(session_data[session_data['session_pitch'] != slowest_pitch]['pitch_speed_mph'])
    
    # Calculate the Z-score for the slowest pitch
    z_score_slowest = (session_data.loc[session_data['pitch_speed_mph'].idxmin()]['pitch_speed_mph'] - mean_speed) / std_speed
    
    # If the slowest pitch is significantly slower (Z-score <= -1), then proceed with analysis
    if z_score_slowest <= -1:
        
        df_filtered = session_data
        
        # Initialize an empty dataframe to store the trimmed data
        df_trimmed = pd.DataFrame()

        # Find unique pitch sessions for the filtered data
        unique_pitches = df_filtered['session_pitch'].unique()

        # Initialize variable to store the length of the shortest time series
        min_length = float('inf')

        # Loop through each unique pitch to trim and crop the data
        for pitch in unique_pitches:
            df_pitch = df_filtered[df_filtered['session_pitch'] == pitch]
    
            # Find the time of the peak leg lift
            peak_leg_lift_time = df_pitch['pkh_time'].iloc[0]
    
            # Trim the series to start at the peak of the leg lift
            df_trim = df_pitch[df_pitch['time'] >= peak_leg_lift_time]
    
            # Update the 'time' variable so it starts from 0 and increments by 1
            df_trim['time'] = range(len(df_trim))
    
            # Update the minimum length
            min_length = min(min_length, len(df_trim))
    
            # Append to the trimmed dataframe
            df_trimmed = pd.concat([df_trimmed, df_trim])

        # Make each time series the same length by trimming to the shortest one
        df_final = pd.DataFrame()
        for pitch in unique_pitches:
            df_pitch = df_trimmed[df_trimmed['session_pitch'] == pitch]
            df_final = pd.concat([df_final, df_pitch.iloc[:min_length]])

        # Show the first few rows of the final trimmed and cropped dataframe
        trimmed_dfs = df_final
            
        scores = {}
        
        for col in trimmed_dfs.columns[3:-1]:
            
            sums =  0
            
            for t in range(0, min_length):
            
                slowest_value = trimmed_dfs[(trimmed_dfs['session_pitch'] == slowest_pitch) & (trimmed_dfs['time'] == t)][col].iloc[0]
                #slowest_value = trimmed_dfs.query(f"session_pitch == '{slowest_pitch}' and time == {t}")[col].iloc[0]
                #print(slowest_value)
                other_values = trimmed_dfs[(trimmed_dfs['session_pitch'] != slowest_pitch) & (trimmed_dfs['time'] == t)][col]
                average_value = sum(other_values)/(len(trimmed_dfs['session_pitch'].unique()) - 1)
                diff = abs(slowest_value - average_value)
                sums += (diff * (min_length - t))
                         
            multiplier = 0.5  # Default multiplier
            if any(keyword in col for keyword in ['ankle', 'knee', 'hip', 'pelvis']):
                multiplier = 1.5
            elif any(keyword in col for keyword in ['torso', 'shoulder']):
                multiplier = 1.0
            
            scores[col] = (sums/min_length) * multiplier
        
        
        top_3_vars = nlargest(3, sample_dict, key=sample_dict.get)
        top_3_items = {k: sample_dict[k] for k in top_3_vars}
        
        root_cause_dict[session] = top_3_items
        

var_frequency = Counter()

for inner_dict in root_cause_dict.values():
    var_frequency.update(inner_dict.keys())

var_frequency
        


In [ ]:
root_cause_dict

In [39]:
max(root_cause_dict, key=root_cause_dict.get)

2889

In [40]:
# Sample code to demonstrate getting the 3 elements in a dictionary with the highest values
from heapq import nlargest

# Sample dictionary
sample_dict = {'a': 1, 'b': 5, 'c': 3, 'd': 4, 'e': 2}

# Get the 3 keys with the highest values
top_3_keys = nlargest(3, sample_dict, key=sample_dict.get)

# Get the 3 highest key-value pairs
top_3_items = {k: sample_dict[k] for k in top_3_keys}

top_3_keys, top_3_items


(['b', 'd', 'c'], {'b': 5, 'd': 4, 'c': 3})